# Phindr3D in python

In [1]:
%matplotlib widget
import phindr_functions as phi
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance as dist

# Preliminary set-up:

In [2]:
param = phi.initParameters()

# EDIT HERE
#change parameters as needed
#Super-Voxel dimensions
param.tileX = 10 
param.tileY = 10
param.tileZ = 3
#Mega-Voxel dimensions
param.megaVoxelTileX = 5
param.megaVoxelTileY = 5
param.megaVoxelTileZ = 2
#Number of categories for binning
param.numVoxelBins = 20
param.numSuperVoxelBins = 15
param.numMegaVoxelBins = 30
#foreground thresholds
param.intensityThresholdTuningFactor = 0.5
param.superVoxelThresholdTuningFactor = 0.5
param.megaVoxelThresholdTuningFactor = 0.5
#training parameters:
param.randTrainingSuperVoxel = 10000
param.pixelsPerImage = 200 
param.randTrainingFields = 5
param.randTrainingPerTreatment = 1
param.intensityNormPerTreatment = False
#visualization:
param.showBincenters = False
param.showImage = False #may be more useful to change this lower down, otherwise may show large number of images
param.showChannels = False # show individual channels instead of rgb images.

#include texture features of Mega Voxel image:
param.textureFeatures = True

#misc. (generally don't need to touch)
param.minQuantileScaling = .5
param.maxQuantileScaling = .5
param.countBackground = False
param.showImage = 0
param.startZPlane = 1
param.endZPlane = 500
param.numRemoveZStart = 1
param.numRemoveZEnd = 1
param.computeTAS = 0
param.trainingPerColumn = False
param.treatmentColNameForNormalization = ''
param.imageTreatments = []
param.allTreatments = []
param.trainingColforImageCategories = []
param.superVoxelPerField = param.randTrainingSuperVoxel//param.randTrainingFields
param.lowerbound = [0, 0, 0]
param.upperbound = [1, 1, 1]
param.numChannels = 3 

#don't need to touch these 
param.svcolormap = phi.random_cmap(map_len=param.numSuperVoxelBins+1)
param.mvcolormap = phi.random_cmap(map_len=param.numMegaVoxelBins+1)


# File loading:
Load folder containing images of interest

NOTE:

If the `analysis_folder_path` is different from the `training_folder_path`, then the Profile Images section requires editing to ensure the images for analysis are loaded properly.

In [15]:
# EDIT HERE
training_folder_path = r"FILE_name"
analysis_folder_path = training_folder_path 
output_file_name = "Phindr3d_output.csv"

#file name key
ID_pos='start'
ID_mark=None
ID_markextra=None
slice_mark='z'
chan_mark='ch'
treat_mark=None
treat_endmark=None

In [16]:
files, imageIDs, treatmentids, idstreatment = phi.get_files(training_folder_path, ID_pos=ID_pos, ID_mark=ID_mark, treat_mark=treat_mark, treat_endmark=treat_endmark, ID_markextra=ID_markextra, slice_mark=slice_mark, chan_mark=chan_mark)
allImageID = imageIDs
#will be empty list if treat_mark is none.
param.imageTreatments = idstreatment
param.treatmentColNameForNormalization = treatmentids
if len(idstreatment) > 0:
    param.allTreatments = np.array(list(treatmentids.keys()), dtype='object')

print('number of images:', len(imageIDs))
print('Treatments found:')
print(param.allTreatments)
#properly calculate number of channels
#slices in first ID:
tmpslices = list(files[imageIDs[0]].keys())
param.numChannels = len(files[imageIDs[0]][tmpslices[0]])

number of images: 540
Treatments found:
['TEV' 'TMYC' 'TMYC-T58A' 'TBcl-2' 'TBcl-XL' 'TSNAIL' 'Tp53-R248W'
 'Tp53-R273H' 'TPI3K' 'TPar']


# Rescale intensities:

want to rescale to between 1 and 0

In [ ]:

param = phi.getScalingFactorforImages(files, allImageID, param)

print('Lowerbounds:', param.lowerbound)
print('Upperbounds:', param.upperbound)


# Threshold images:
get threshold value to use from training data


In [ ]:

param = phi.getImageThresholdValues(files, allImageID, param)
intensityThreshold = np.quantile(param.intensityThresholdValues, param.intensityThresholdTuningFactor, axis=0)
param.intensityThreshold = np.reshape(intensityThreshold, (1, param.numChannels))

print(param.intensityThreshold)

# Cluster pixels:
compute pixel categories 

In [ ]:

param = phi.getPixelBinCenters(files, allImageID, param)
print(param.pixelBinCenters) #20 different pixel categories made up of scaled intensities of different channels
print(param.pixelBinCenters.shape)

In [ ]:
#analyze voxel bin centers
# cosine distance to get measure of proportionality between vectors
voxel_sim = 1 - dist.squareform(dist.pdist(param.pixelBinCenters, 'cosine'))
voxel_sim = np.tril(voxel_sim, -1)
mean = np.average(voxel_sim[np.nonzero(voxel_sim)])
std = np.std(voxel_sim[np.nonzero(voxel_sim)])
print(f'Mean voxel bin center similarity: {mean}')
print(f'standard deviation: {std}')
plt.figure()
plt.imshow(voxel_sim)
plt.title('Voxel bin center cosine similarity')
plt.xticks([i for i in range(param.numVoxelBins)], [f'{i+1}' for i in range(param.numVoxelBins)])
plt.yticks([i for i in range(param.numVoxelBins)], [f'{i+1}' for i in range(param.numVoxelBins)])
plt.colorbar()
plt.show()

# Define super voxels:
use pixel categories to make some supervoxels! (also cluster super voxels into bins/categories)

In [ ]:

param = phi.getSuperVoxelBinCenters(files, allImageID, param)
print(param.supervoxelBincenters) #15 different supervoxel categories made up of frequencies of 20 different pixel categories
print(param.supervoxelBincenters.shape)

In [ ]:
# analyze SV bin centers
voxel_sim = 1 - dist.squareform(dist.pdist(param.supervoxelBincenters, 'cosine'))
voxel_sim = np.tril(voxel_sim, -1)
mean = np.average(voxel_sim[np.nonzero(voxel_sim)])
std = np.std(voxel_sim[np.nonzero(voxel_sim)])
print(f'Mean super voxel bin center similarity: {mean}')
print(f'standard deviation: {std}')
plt.figure()
plt.imshow(voxel_sim)
plt.title('Super voxel bin center cosine similarity')
plt.xticks([i for i in range(param.numSuperVoxelBins)], [f'{i+1}' for i in range(param.numSuperVoxelBins)])
plt.yticks([i for i in range(param.numSuperVoxelBins)], [f'{i+1}' for i in range(param.numSuperVoxelBins)])
plt.colorbar()
plt.show()

# Combine super voxels to mega voxels:
use super voxel categories and tileprofile to get megavoxels. also get megavoxel bins/categories

In [ ]:

param = phi.getMegaVoxelBinCenters(files, allImageID, param)
print(param.megaVoxelBincenters)
print(param.megaVoxelBincenters.shape)

In [ ]:
# analyze MV bin centers
voxel_sim = 1 - dist.squareform(dist.pdist(param.megaVoxelBincenters, 'cosine'))
voxel_sim = np.tril(voxel_sim, -1)
mean = np.average(voxel_sim[np.nonzero(voxel_sim)])
std = np.std(voxel_sim[np.nonzero(voxel_sim)])
print(f'Mean mega voxel bin center similarity: {mean}')
print(f'standard deviation: {std}')
plt.figure()
plt.imshow(voxel_sim)
plt.title('Mega voxel bin center cosine similarity')
plt.xticks([i for i in range(0, param.numMegaVoxelBins, 2)], [f'{i+1}' for i in range(0, param.numMegaVoxelBins, 2)])
plt.yticks([i for i in range(0, param.numMegaVoxelBins, 2)], [f'{i+1}' for i in range(0, param.numMegaVoxelBins, 2)])
plt.colorbar()
plt.show()

# Profile images:


In [ ]:
# to use previously calculated categories, uncomment lines below to specify different location of files 

# to be analyzed: EDIT HERE
# folder_path =  r'E:\Phindr3D_screenC\screenC\plate2' 

###
# files, imageIDs, treatmentids, idstreatment = phi.get_files(analysis_folder_path, ID_pos=ID_pos, ID_mark=ID_mark, treat_mark=treat_mark, treat_endmark=treat_endmark, ID_markextra=ID_markextra, slice_mark=slice_mark, chan_mark=chan_mark)
# param.imageTreatments = idstreatment
# param.treatmentColNameForNormalization = treatmentids
# if len(idstreatment) > 0:
#     param.allTreatments = np.array(list(treatmentids.keys()), dtype='object')
# tmpslices = list(files[imageIDs[0]].keys())
# param.numChannels = len(files[imageIDs[0]][tmpslices[0]])
# allImageID = imageIDs
###

%matplotlib inline
#Set to True to show images in process (will show LARGE number of images): EDIT HERE
param.showImage = True 

param, resultIM, resultRaw, df = phi.extractImageLevelTextureFeatures(files, allImageID, param, outputFileName=output_file_name, outputDir='')

from IPython.display import display
display(df)